# Test BSM model implementation
Repeat the same for normal model implementation

In [18]:
import numpy as np

In [19]:
# import
from option_models import bsm
from option_models import normal

In [20]:
### only run this when you changed the class definition
import imp
imp.reload(bsm)
imp.reload(normal)

<module 'option_models.normal' from 'C:\\Users\\Noam\\Documents\\GitHub\\PHBS_ASP_2018\\HW3\\option_models\\normal.py'>

### Price

In [21]:
# create model
bsm1 = bsm.BsmModel(0.2)

In [22]:
# price
strike = 102
spot = 100
texp = 0.25

price = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
print(price)
assert( abs(price - 3.10628366655) < 1e-10 )

3.1062836665495652


### Implied vol

In [12]:
# Randomly generate spot/strike/expiry/intr/divr/cp_sign
# Then test implied volatility

for k in range(100):
    spot = np.random.uniform(80,100)
    strike = np.random.uniform(80,100)
    vol = np.random.uniform(0.0001, 0.4)
    texp = np.random.uniform(0.1, 5)
    intr = np.random.uniform(0, 0.3)
    divr = np.random.uniform(0, 0.3)
    cp_sign = 1 if np.random.rand() > 0.5 else -1

    #print( spot, strike, vol, texp, intr, divr, cp_sign)

    bsm2 = bsm.BsmModel(vol=vol, intr=intr, divr=divr)
    price = bsm2.price(strike, spot, texp, cp_sign )
    
    # get implied vol
    vol_imp = bsm2.impvol(price, strike, spot, texp=texp, cp_sign=cp_sign)
    
    # now price option with the obtained implied vol
    bsm2.vol = vol_imp
    price_imp = bsm2.price(strike, spot, texp, cp_sign )
    
    # compare the two prices
    assert( abs(price - price_imp) < 1e-8 )

### Delta
Verify delta by comparing numerical derivative

In [28]:
h = 1e-6
vol=0.2

In [30]:
delta = bsm1.delta(strike=102, spot=100, texp=0.25, cp_sign=1)
price_up = bsm1.price(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
price_down = bsm1.price(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
delta_numeric = (price_up - price_down)/(2*h)
print(delta, delta_numeric)

0.4411610169119095 0.4411610170507174


In [26]:
assert(abs(delta - delta_numeric) < 1e-6)

### Vega
Do similar verification

In [38]:
texp=0.25
vol=0.2

In [43]:
vega = bsm1.vega(strike, spot, texp, cp_sign=1)
price_up = bsm.bsm_price(strike=strike, spot=spot, vol=vol+h, texp=texp, cp_sign=1)
price_down = bsm.bsm_price(strike=strike, spot=spot, vol=vol-h, texp=texp, cp_sign=1)
vega_numeric = (price_up - price_down)/(2*h)

print(vega, vega_numeric)

TypeError: vega() missing 1 required positional argument: 'texp'

In [44]:
assert(abs(vega - vega_numeric) < 1e-6)

NameError: name 'vega' is not defined

### Gamma
Do similar verification

In [45]:
gamma = bsm1.gamma(strike, spot, texp, cp_sign=1)

price_up = bsm1.price(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
price = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
price_down = bsm1.price(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
gamma_numeric = ( price_up - 2*price + price_down )/(h**2)

print(gamma, gamma_numeric)

TypeError: gamma() missing 1 required positional argument: 'texp'

In [ ]:
assert(abs(gamma - gamma_numeric) < 1e-6)